## Import Modules

In [1]:
import numpy as np
import pandas as pd
from numba import jit
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, KFold
import seaborn as sns

import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt


import os
import warnings
warnings.filterwarnings('ignore')

## Import Data

In [2]:
os.chdir('C:/Kaggle')

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
features = [c for c in train.columns if c not in ['ID_code', 'target']]
target = train['target']

## Augment Function

In [3]:
def augment(x,y,t=2):
    xs,xn = [],[]
    for i in range(t):
        mask = y>0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xs.append(x1)

    for i in range(t//2):
        mask = y==0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xn.append(x1)

    xs = np.vstack(xs)
    xn = np.vstack(xn)
    ys = np.ones(xs.shape[0])
    yn = np.zeros(xn.shape[0])
    x = np.vstack([x,xs,xn])
    y = np.concatenate([y,ys,yn])
    return x,y

## Hyper Parameters

In [4]:
param = {
    'bagging_freq': 5,
    'bagging_fraction': 0.335,
    'boost_from_average':'false',
    'boost': 'gbdt',
    'feature_fraction': 0.041,
    'learning_rate': 0.0083,
    'max_depth': -1,
    'metric':'auc',
    'min_data_in_leaf': 80,
    'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 13,
    'num_threads': 8,
    'tree_learner': 'serial',
    'objective': 'binary',
    'verbosity': -1
}

## LightGBM with K-folds CV

In [5]:
num_folds = 11
features = [c for c in train.columns if c not in ['ID_code', 'target']]

folds = KFold(n_splits=num_folds, random_state=2319)
oof = np.zeros(len(train))
getVal = np.zeros(len(train))
predictions = np.zeros(len(target))
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
    X_train, y_train = train.iloc[trn_idx][features], target.iloc[trn_idx]
    X_valid, y_valid = train.iloc[val_idx][features], target.iloc[val_idx]

    X_tr, y_tr = augment(X_train.values, y_train.values)
    X_tr = pd.DataFrame(X_tr)

    print("Fold idx:{}".format(fold_ + 1))
    trn_data = lgb.Dataset(X_tr, label=y_tr)
    val_data = lgb.Dataset(X_valid, label=y_valid)

    clf = lgb.train(param, trn_data, 1000000, valid_sets=[trn_data, val_data], verbose_eval=5000,
                    early_stopping_rounds=4000)
    oof[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    getVal[val_idx] += clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration) / folds.n_splits

    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

    predictions += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits

Fold idx:1
Training until validation scores don't improve for 4000 rounds.
[5000]	training's auc: 0.912493	valid_1's auc: 0.896426
[10000]	training's auc: 0.922472	valid_1's auc: 0.900592
[15000]	training's auc: 0.930026	valid_1's auc: 0.901593
Early stopping, best iteration is:
[15077]	training's auc: 0.930129	valid_1's auc: 0.901607
Fold idx:2
Training until validation scores don't improve for 4000 rounds.
[5000]	training's auc: 0.912774	valid_1's auc: 0.895506
[10000]	training's auc: 0.922635	valid_1's auc: 0.898671
[15000]	training's auc: 0.93015	valid_1's auc: 0.899083
Early stopping, best iteration is:
[12546]	training's auc: 0.926611	valid_1's auc: 0.899228
Fold idx:3
Training until validation scores don't improve for 4000 rounds.
[5000]	training's auc: 0.912639	valid_1's auc: 0.88926
[10000]	training's auc: 0.922486	valid_1's auc: 0.892514
[15000]	training's auc: 0.930006	valid_1's auc: 0.892616
Early stopping, best iteration is:
[13436]	training's auc: 0.927745	valid_1's auc: 

## Submmision

In [6]:
submission = pd.DataFrame({"ID_code": test.ID_code.values})
submission["target"] = predictions
submission.to_csv("submission_new_lgbm.csv", index=False)

submission.head()

,ID_code,target
0,test_0,0.143631
1,test_1,0.267358
2,test_2,0.225609
3,test_3,0.290507
4,test_4,0.060658
